<b>Определение NPS среди клиентов из России </b>

Задачи проекта: Исследование опроса клиентов телекомунникацонной компании.
Подключимся к файлу базы данных. Иследуем его, соединим в нужный нам датафрейм. И уже его будем визуализировать и анализировать в Tableau.


In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)

In [2]:
path_to_db_local = '......'
path_to_db = None

In [3]:
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
else:
    raise Exception('Файл с базой данных SQLite не найден!')
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    # возьмем сразу перечислим в select нужные поля
    # Используем оператор CASE для столбцов is_new, gender_segment, nps_score
    query1 = """
    SELECT *
    FROM location;
    """
    query2 = """
    SELECT *
    FROM age_segment;
    """
   
    query3 = """
    SELECT *
    FROM traffic_segment;
    """
    query4 = """
    SELECT *
    FROM lifetime_segment;
    """
    query5 = """
    SELECT *
    FROM user;
    """
    
    # создаём датафрейм по данным запроса
    df1 = pd.read_sql(query1, engine)
    df2 = pd.read_sql(query2, engine)
    df3 = pd.read_sql(query3, engine)
    df4 = pd.read_sql(query4, engine)
    df5 = pd.read_sql(query5, engine)

In [5]:
df1.info() # Дата сет с локацией.
df1
# Нет пустых ячеек - Страна - Россия, в city - перечислены города

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  62 non-null     int64 
 1   city         62 non-null     object
 2   country      62 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия
...,...,...,...
57,58,Челябинск,Россия
58,59,Череповец,Россия
59,60,Чита,Россия
60,61,Якутск,Россия


In [6]:
df2.info() # датасет с возрастом
df2
# Датасет с возрастом. Есть пустая ячейка в bucket_min при 1 группе возраста (от 1 до 16 лет).
# И есть группа 08 - которая наверног не указала свой возраст при опросе.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age_gr_id   8 non-null      int64  
 1   bucket_min  6 non-null      float64
 2   bucket_max  6 non-null      float64
 3   title       8 non-null      object 
dtypes: float64(2), int64(1), object(1)
memory usage: 384.0+ bytes


,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


In [7]:
df3.info()# Дата сет с объемом потребляемого трафика.
df3
# 24 группа в bucket_max стоит пустое значение. Что логично от 100 до бесконечности.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tr_gr_id    25 non-null     int64  
 1   bucket_min  25 non-null     float64
 2   bucket_max  24 non-null     float64
 3   title       25 non-null     object 
dtypes: float64(2), int64(1), object(1)
memory usage: 928.0+ bytes


,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


In [8]:
df4.info() # Датасет  с месяцев жизни клиента
df4
# Также есть пустое значение, что логично,  в 8 группе от 36 минут до бесконечности....

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   lt_gr_id    8 non-null      int64  
 1   bucket_min  8 non-null      float64
 2   bucket_max  7 non-null      float64
 3   title       8 non-null      object 
dtypes: float64(2), int64(1), object(1)
memory usage: 384.0+ bytes


,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


In [9]:
df5.info() # Основная таблица с информацией о клиентах
df5
# Есть пустые ячейки в столбцах: age,  gender_segment.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 42.2+ MB


,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10
...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,54.0,1.0,ANDROID,SMARTPHONE,28,5,5,8,5
502489,ZZZLWY,129,31.0,0.0,ANDROID,SMARTPHONE,28,3,5,4,8
502490,ZZZQ5F,522,36.0,0.0,ANDROID,SMARTPHONE,47,4,10,6,10
502491,ZZZQ8E,2936,37.0,1.0,ANDROID,SMARTPHONE,53,4,18,8,9


In [10]:
display(df5.age.unique())
display(df5.age.describe())

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

count    501939.000000
mean         39.621946
std          11.188249
min          10.000000
25%          31.000000
50%          38.000000
75%          47.000000
max          89.000000
Name: age, dtype: float64

In [11]:
df5.gender_segment.unique()

array([ 1.,  0., nan])

In [12]:
print('Количество пустых значений:',df5.gender_segment.isna().sum())
print('Процент этих значений от общего количества:',round((df5.gender_segment.isna().sum()) * 100 /len(df5), 2),'%')
# Пустых значений 1301 из 502493. Что составляет 0.26 %. 

Количество пустых значений: 1301
Процент этих значений от общего количества: 0.26 %


Могли бы эти значения удалить в датасете 0,2% это очень мало, но по условиям задачи нам не надо проводить предобработку данных

In [15]:
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
else:
    raise Exception('Файл с базой данных SQLite не найден!')
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    # возьмем сразу перечислим в select нужные поля
    # Используем оператор CASE для столбцов is_new, gender_segment, nps_score
    query = """
    SELECT
        u.user_id,
        u.lt_day,
        (CASE WHEN u.lt_day < 365 THEN 'New' ELSE 'Old' END) AS is_new, 
        u.age,
        (CASE WHEN u.gender_segment = 0.0 THEN 'Мужчина' 
              WHEN u.gender_segment = 1.0 THEN 'Женщина' END) AS gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        ag.title AS age_segment,
        ts.title AS traffic_segment,
        ls.title AS lifetime_segment,
        u.nps_score,
        (CASE WHEN (u.nps_score = 9) or (u.nps_score = 10) THEN 'promoters' 
              WHEN (u.nps_score = 7) or (u.nps_score = 8) THEN 'passives'
              ELSE 'detractors' 
        END) AS nps_group

    FROM user AS u
    LEFT OUTER JOIN location as l ON l.location_id = u.location_id
    LEFT OUTER JOIN age_segment as ag ON ag.age_gr_id = u.age_gr_id
    LEFT OUTER JOIN traffic_segment as ts ON ts.tr_gr_id = u.tr_gr_id
    LEFT OUTER JOIN lifetime_segment as ls ON ls.lt_gr_id = u.lt_gr_id;
    """

    # создаём датафрейм по данным запроса
    df = pd.read_sql(query, engine)

In [16]:
display(df)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Old,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,Old,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,Old,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,Old,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,Old,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,Old,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Москва,05 45-54,04 1-5,08 36+,5,detractors
502489,ZZZLWY,129,New,31.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,03 25-34,04 1-5,04 4-6,8,passives
502490,ZZZQ5F,522,Old,36.0,Мужчина,ANDROID,SMARTPHONE,Россия,Сургут,04 35-44,09 25-30,06 13-24,10,promoters
502491,ZZZQ8E,2936,Old,37.0,Женщина,ANDROID,SMARTPHONE,Россия,УланУдэ,04 35-44,17 65-70,08 36+,9,promoters


In [17]:
# Напишем функцию на вывод описательную статистикку для датафрейма
def data_opisanie(data):
    return data.head(2), data.tail(2), data.describe(), data.info()

In [18]:
display(data_opisanie(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


(  user_id  lt_day is_new   age gender_segment  os_name cpe_type_name country  \
 0  A001A2    2320    Old  45.0        Женщина  ANDROID    SMARTPHONE  Россия   
 1  A001WF    2344    Old  53.0        Мужчина  ANDROID    SMARTPHONE  Россия   
 
     city age_segment traffic_segment lifetime_segment  nps_score  nps_group  
 0    Уфа    05 45-54          04 1-5           08 36+         10  promoters  
 1  Киров    05 45-54          04 1-5           08 36+         10  promoters  ,
        user_id  lt_day is_new   age gender_segment  os_name cpe_type_name  \
 502491  ZZZQ8E    2936    Old  37.0        Женщина  ANDROID    SMARTPHONE   
 502492  ZZZYH0    2471    Old  37.0        Мужчина      IOS    SMARTPHONE   
 
        country     city age_segment traffic_segment lifetime_segment  \
 502491  Россия  УланУдэ    04 35-44        17 65-70           08 36+   
 502492  Россия   Самара    04 35-44        06 10-15           08 36+   
 
         nps_score   nps_group  
 502491          9   promot

In [19]:
df.gender_segment.unique()

array(['Женщина', 'Мужчина', None], dtype=object)

In [20]:
df.is_new.unique()

array(['Old', 'New'], dtype=object)

In [21]:
# Сохраним в csv, столбец индекса сохранять не будем, так как по условию задачи поле индекса нам не нужно
df.to_csv('3.csv', index=False)

Ссылка на tableau https://public.tableau.com/views/NPS_16899531532420/Dashboard2?:language=en-US&:display_count=n&:origin=viz_share_link